<a href="https://colab.research.google.com/github/CatMcQueen/ECE523_ML/blob/main/Adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
class Adaboost():
  def __init__(self):
    #self.weights = None
    self.alpha = []
    #self.error  = []
    self.model = []
    self.rounds = 0


  def sample(self, N, p):
    # this is the sample weak model
    # from homework 1
    random_sample = np.zeros(N)
    p_estimate = np.zeros(len(p))
    p_cdf = np.cumsum(p)
    counts = np.zeros(len(p))

    for n in range(N):
        # generate a random number on [0,1]
        x = np.random.rand()
        random_sample[n] = int(np.where(((p_cdf > x)*1.0) == 1.)[0][0])
        counts[int(random_sample[n])] += 1

    p_estimate = counts/counts.sum()
    random_sample = [int(x) for x in random_sample]
    return random_sample, p_estimate

  def calc_error(self, weights,  y, y_pred):
    # error = sum(weights* [[h(i) != y(i)]])
    # cheat and use np not equal to do the [[]] part becuase otherwise it would 
    # be a super sucky for loop that would take forever
    # also should I be normalizing the error? I think weights
    # should always sum to 1???
    not_equal_part  = np.not_equal(y, y_pred)
    err             = sum(weights * not_equal_part.astype(int))/sum(weights)
    return err

  def calc_alpha(self, error):
    # alpha = 1/2*log((1-error)/error)
    # np.log is the ln which is what we are using in class
    alpha = 0
    if(error != 0):
      alpha = .5 * np.log((1-error)/error)
    return alpha

  def calc_weight(self, prev_weight, y, alpha, y_pred, error):
    # Di = Di/Z * exp(-alpha * y * y_pred)
    # Z = 1 if z = true and z=0 if false
    #z  = np.not_equal(y, y_pred)
    #zp = np.equal(y, y_pred)
    zt = (1-error)*np.exp(-alpha) + (error)*np.exp(alpha)
    new_weights = prev_weight / np.sum(zt) * np.exp(-alpha * y * y_pred)
    return new_weights



  def fit(self, x, y, rounds=20):
    weights = np.ones(len(x))/len(x)
    #self.weights.append(weights)
    self.rounds = rounds

    for n in range(rounds):
      # first fit the weak model with a sample of the values
      weakmodel = DecisionTreeClassifier(max_depth=2)
      i, p      = self.sample(len(weights), weights)
      weakmodel = weakmodel.fit(x[i], y[i])

      # then do the steps from the lecture (4)
      y_pred    = weakmodel.predict(x)
      error     = self.calc_error(weights, y, y_pred)
      alpha     = self.calc_alpha(error)
      weights   = self.calc_weight(weights, y, alpha, y_pred, error)
      #print(weights)

      # now save off relevant values
      #self.weights.append(weights) # doesn't look like we need these
      #self.error.append(error) # doesn't look like we need
      self.alpha.append(alpha)
      self.model.append(weakmodel)
    #yhat = sign(sum(alpha* y_pred(x)))
    # accuracy is number of that that's right
    #return accuracy?? I dont think I need to return anything?

  def predict(self, x):
    ## predict is y_hat = sign (sum(alpha * y_pred(x) ))
    sumval = []
    for n in range(self.rounds):
      # then do the weak prediction
      y_pred    = self.model[n].predict(x) * self.alpha[n]
      sumval.append(y_pred)

    weak_pred = np.array(sumval)

    ## predict is y_hat = sign (sum(alpha * y_pred(x) ))
    vals = np.sum(weak_pred, axis=0)
    #print(len(vals))
    #print(np.sign(vals))
    y_hat = np.sign(np.sum(weak_pred, axis=0)).astype(int)
    return y_hat


In [3]:
def data_process(pd_source, pd_target):
  # convert data and split it into x and y
  sourcearray = np.array(pd_source)
  targetarray = np.array(pd_target)
  target_x = targetarray[:, :-1] # the x1 and x2 cols
  target_y = targetarray[:, -1]  # the y column
  source_x = sourcearray[:, :-1] # the x1 and x2 cols
  source_y = sourcearray[:, -1]  # the y column

  # now convert it to "matrix" form?
  xt = np.array(target_x)
  yt = np.array(target_y)
  xs = np.array(source_x)
  ys = np.array(source_y)

  return xs, ys, xt, yt

In [5]:
import pandas as pd
# first collect the source data
pd_target = pd.read_csv('target_train.csv', header=None,  dtype=float)
pd_source = pd.read_csv('source_train.csv', header=None,  dtype=float)

# now separate it into x and y for the source and target
xs, ys, xt, yt = data_process(pd_source, pd_target)


In [ ]:
xs.shape

In [6]:
ada = Adaboost()
ada.fit(xs, ys)

In [7]:
predictions = ada.predict(xt)

In [8]:
print(predictions)
print(yt)

[ 1 -1  1 -1 -1  1  1  1 -1 -1 -1  1  1 -1  1  1  1  1  1 -1  1  1  1 -1
 -1  1  1 -1  1 -1  1 -1  1  1  1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1  1
 -1 -1]
[ 1. -1.  1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1. -1.  1.  1.  1.  1.
  1. -1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1. -1.
  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1.  1. -1. -1.]


In [9]:
count = 0
for i, x in enumerate(predictions):
  if (x == int(yt[i])):
    count += 1

print (count / len(predictions))

0.98


In [10]:
from sklearn.ensemble import AdaBoostClassifier


clf = AdaBoostClassifier(n_estimators=30, random_state=0)
clf.fit(xs, ys)
sklearn_ada = clf.predict(xt)

In [11]:
count = 0
for i, x in enumerate(sklearn_ada):
  if (x == int(yt[i])):
    count += 1

print (count / len(predictions))

0.98
